In [1]:
# Dependencies

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pickle

from DataSource import DataSource
from cnn_lstm import CNNLSTMModel

c:\Users\qrnts\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
# Options
trend_length = 5
cnn_length = 200
lstm_length = 60
symbol = "ETHUSDT"
start_time = '2024-03-16'
end_time = '2024-05-17'
interval = '1h'
batch_size = 32
ohclv_length = 5
num_lstm_layers = 8
cnn_embedding_length = 128
lstm_hidden_dim = 64

dataLoader = {
    "trend_length": trend_length,
    "cnn_length": cnn_length,
    "lstm_length": lstm_length,
    "symbol": symbol,
    "start_time": start_time,
    "end_time": end_time,
    "interval": interval,
    "batch_size": batch_size
}


In [4]:
# Save Data
data = DataSource(**dataLoader)

with open("./trained_Data", 'wb') as f:
    pickle.dump(data, f)


In [5]:
# Load Data
with open('./trained_Data', 'rb') as f:
    data = pickle.load(f)

In [6]:
# Initialize Model

model = CNNLSTMModel(
    cnn_length = cnn_length, 
    cnn_embedding_length = cnn_embedding_length, 
    ohclv_length = ohclv_length, 
    lstm_hidden_dim = lstm_hidden_dim, 
    output_dim= 1,
    num_lstm_layers= num_lstm_layers
).to(device)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.1)
# optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
# Train
epochs = 100
inputLoader = data.cnn_data
labelLoader = data.label_data
n_batch = len(inputLoader)

for epoch in range(epochs):  # loop over the dataset multiple times. You can increase if you want.
    epoch_loss = 0.0
    
    for batch in range(n_batch):
        if(batch_size != inputLoader[batch].shape[0]): continue
        if(batch_size != labelLoader[batch].shape[0]): continue
        inputs = torch.tensor(inputLoader[batch], dtype=torch.float32).to(device)
        labels = torch.tensor(labelLoader[batch], dtype=torch.float32).to(device)
        ohclv = torch.tensor(data.lstm_data[batch], dtype=torch.float32).to(device)
        optimizer.zero_grad()
        output = model(inputs, ohclv)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    print('[%d] loss: %.5f' %(epoch + 1, epoch_loss/batch_size))

[1] loss: 0.90963
[2] loss: 0.90958
[3] loss: 0.90954
[4] loss: 0.90950
[5] loss: 0.90945
[6] loss: 0.90941
[7] loss: 0.90937
[8] loss: 0.90934
[9] loss: 0.90930
[10] loss: 0.90926
[11] loss: 0.90923
[12] loss: 0.90919
[13] loss: 0.90916
[14] loss: 0.90913
[15] loss: 0.90909
[16] loss: 0.90906
[17] loss: 0.90903
[18] loss: 0.90900
[19] loss: 0.90898
[20] loss: 0.90895
[21] loss: 0.90892
[22] loss: 0.90890
[23] loss: 0.90887
[24] loss: 0.90884
[25] loss: 0.90882
[26] loss: 0.90880
[27] loss: 0.90877
[28] loss: 0.90875
[29] loss: 0.90873
[30] loss: 0.90871
[31] loss: 0.90869
[32] loss: 0.90867
[33] loss: 0.90865
[34] loss: 0.90863
[35] loss: 0.90861
[36] loss: 0.90860
[37] loss: 0.90858
[38] loss: 0.90856
[39] loss: 0.90855
[40] loss: 0.90853
[41] loss: 0.90852
[42] loss: 0.90850
[43] loss: 0.90849
[44] loss: 0.90847
[45] loss: 0.90846
[46] loss: 0.90845
[47] loss: 0.90843
[48] loss: 0.90842
[49] loss: 0.90841
[50] loss: 0.90840
[51] loss: 0.90838
[52] loss: 0.90837
[53] loss: 0.90836
[5